# Introduction

In this notebook, different methods are used to solve the linear system $Ax = b$ where \\

\begin{equation*}
A = \begin{bmatrix}
0 & 3 & -1 & 8 \\
-1 & 11 & -1 & 3 \\
2 & -1 & 10 & - 1 \\
10 & -1 & 2 & 0
\end{bmatrix} \\
\end{equation*}

\begin{equation*}
b = \begin{bmatrix}
15 \\
25 \\
-11 \\
6
\end{bmatrix}
\end{equation*} \\

All the methods are implemented from scratch without using any linear algebra package. First, let's import relevant libraries.

In [1]:
import numpy as np
import itertools

Then we create the matrix $A$ and the vector $b$ as numpy arrays. A variable $n$ is created to store the number of rows (and columns as $A$ is a square matrix) of the matrix. 

In [2]:
A = np.array([[0., 3., -1., 8.], [-1., 11., -1., 3.], [2., -1., 10., -1.], [10., -1., 2., 0.]]) # matrix
b = np.array([15., 25., -11., 6.]) # vector

n = np.shape(A)[0] # number of rows and columns 

# Strictly diagonally dominant

As a first step, we want to put the matrix $A$ in a strictly diagonally dominant form (if possible). A matrix \textbf{M} is strictly diagonally dominant if 

\begin{equation*} 
\left|\bf{M}\right|_{ii} > \sum_{i \neq j} \left|\bf{M}\right|_{ij}
\end{equation*} \\

In order to check if the matrix $A$ is strictly diagonally dominant (SDD), a function is created. The function takes as inputs the matrix $A$ and the vector $b$. The function switches the rows of the input matrix and checks if the matrix with switched rows is SDD. When it finds a combination of rows that makes the matrix SDD, it returns the matrix with this combination of rows and the corresponding vector. If none of the combinations makes the matrix SDD, the input matrix and vector are returned. 

In [3]:
def is_SDD(A, b):
    check = False
    n = np.shape(A)[0] # number of rows in matrix A
    perm_list = list(itertools.permutations(np.arange(n)))
    for perm in perm_list:                            
        mat = A[perm,:] # matrix with switched rows
        
        # Check if the matrix is SDD
        for y in range(n):
            my_sum = 0
            for z in range(n):
                if y == z:
                    my_sum += np.abs(mat[y, z]) # add when on diagonal
                else:
                    my_sum -= np.abs(mat[y, z]) # substract if not on diagonal
            if my_sum <= 0:
                check = False
                break # avoid unnecessary computation if current mat is not SDD
            else: 
                check = True
        
        # If matrix is SDD, return the new matrix with switched rows
        # and corresponding vector
        if check == True:
            vec = [x for y, x in sorted(zip(perm, b))] # switch rows of the vector
            print('The returned matrix is strictly diagonally dominant. \n')
            print('The returned matrix is \n')
            print(mat)
            print('\n The returned vector is \n')
            print(vec)
            return mat, vec
   
    # If matrix is not SDD, return the original matrix and vector
    print('The input matrix cannot be put in striclty diagonally dominant form.')
    print('The original matrix and vector are returned.')
    return A, b  

We now check if the matrix $A$ can be put in SDD form.

In [4]:
A, b = is_SDD(A, b)

The returned matrix is strictly diagonally dominant. 

The returned matrix is 

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]

 The returned vector is 

[6.0, 25.0, -11.0, 15.0]


Now that the matrix $A$ is in strictly diagonally dominant form, we are going to solve the system $Ax = b$ for $x$ using different methods. 

# LU Decomposition

## Doolittle's method 


The first step is to decompose the matrix $A$ into a lower triangular matrix, $L$, and an upper triangular matrix, $U$ such that $A = LU$. With Doolittle's method, the matrix $L$ is constructed such that $L_{ii} = 1 \,\, \forall \,\,1 \leq i \leq n$.

We now decompose the matrix $A$ into $L$ and $U$.

In [5]:
# Initialize the matrices 
L = np.zeros((n, n))
U = np.zeros((n, n))

# LU decomposition
for i in range(n):
    # Upper triangular matrix
    for k in range(i, n):
        my_sum = 0
        for j in range(i):
            my_sum += L[i, j] * U[j, k]
            
        U[i, k] = A[i, k] - my_sum
    
    # Lower triangular matrix
    for k in range(i, n):
        if i == k:
            L[i, k] = 1 # 1 on diagonal
        else:
            my_sum = 0
            for j in range(i):
                my_sum += L[k, j] * U[j, i]
                
            L[k, i] = (A[k, i] - my_sum) / U[i, i]

print('The matrix L is \n')
print('L =', np.array2string(L, prefix='L = '))
print('\n The matrix U is \n')
print('U =', np.array2string(U, prefix='U = '))

The matrix L is 

L = [[ 1.          0.          0.          0.        ]
     [-0.1         1.          0.          0.        ]
     [ 0.2        -0.0733945   1.          0.        ]
     [ 0.          0.27522936 -0.08173077  1.        ]]

 The matrix U is 

U = [[10.         -1.          2.          0.        ]
     [ 0.         10.9        -0.8         3.        ]
     [ 0.          0.          9.5412844  -0.77981651]
     [ 0.          0.          0.          7.11057692]]


A quick check shows that $A = LU$ is satisfied.

In [6]:
print('LU =', np.array2string(np.dot(L,U), prefix='LU = '), '\n\n','A =', np.array2string(A, prefix='A =  '))

LU = [[10. -1.  2.  0.]
      [-1. 11. -1.  3.]
      [ 2. -1. 10. -1.]
      [ 0.  3. -1.  8.]] 

 A = [[10. -1.  2.  0.]
      [-1. 11. -1.  3.]
      [ 2. -1. 10. -1.]
      [ 0.  3. -1.  8.]]


The system $Ax = b$ can be written as $Ax = (LU)x = L(Ux) = b$. Introducing the vector $z = Ux$, we have $Lz = b$. 

We now solve for $z$ using forward substitution.

In [7]:
# Initialize array for the vector z
z = np.zeros(n)
z[0] = b[0]

# Solve Lz = b for z using forward substitution
for i in range(1, n):
    my_sum = 0
    for j in range(i):
        my_sum += L[i, j] * z[j]
    z[i] = b[i] - my_sum

print('The vector z is \n')
print('z =', z)

The vector z is 

z = [  6.          25.6        -10.32110092   7.11057692]


Again, a quick check shows that $Lz = b$ is satisfied.

In [8]:
print('Lz =', np.dot(L, z), '\n\n', 'b =', b)

Lz = [  6.  25. -11.  15.] 

 b = [6.0, 25.0, -11.0, 15.0]


Now that $z$ is known, the last step is to solve $Ux = z$ for $x$ using backward substitution.

In [9]:
# Initialize array for the vector x
x = np.zeros(n)
x[n-1] = z[n-1]/U[n-1, n-1]

# Solve Ux = z for x using backward substitution
for i in range(n-2, -1, -1):
    my_sum = 0
    for j in range(n-1, i, -1):
        my_sum = my_sum + U[i, j] * x[j]
    x[i] = (z[i] - my_sum) / U[i, i]

print('The vector x is \n')
print('x =', x)

The vector x is 

x = [ 1.  2. -1.  1.]


A quick check shows that $Ax = b$ is satisfied.

In [10]:
print('Ax =', np.dot(A, x), '\n\n', 'b =', b, '\n\n', np.dot(A, x)==b)

Ax = [  6.  25. -11.  15.] 

 b = [6.0, 25.0, -11.0, 15.0] 

 [ True  True  True  True]


The last line shows that Doolittle's method is very accurate. The matrix vector product $Ax$ returns exactly the value of $b$.

## Crout's method

Crout's method is very similar to Doolittle's method. The difference is that the LU decomposition is done such that there is a diagonal of ones in the upper triangular matrix whereas the diagonal of ones is in the lower triangular matrix with Doolittle's method. Hence with Crout's method, $U_{ii} = 1 \,\, \forall \,\, 1 \leq i \leq n$.

We now decompose $A$ into $L$ and $U$.

In [11]:
# Initialize the matrices
L = np.zeros((n, n))
U = np.zeros((n, n))

# LU decomposition
for i in range(0, n):
    # Lower triangular matrix
    for k in range(i, n):
        my_sum = 0
        for j in range(0, i):
            my_sum += L[i, j] * U[j, k] 
        L[k, i] = A[i, k] - my_sum
    
    # Upper triangular matrix
    for k in range(i, n):
        if i == k:
            U[i, k] = 1 # 1 on the diagonal
        else:
            my_sum = 0
            for j in range(0, i):
                my_sum += L[i, j] * U[j, k] 
            U[i, k] = (A[i, k] - my_sum) / L[i, i]
            
print('The matrix L is \n')
print('L =', np.array2string(L, prefix='L = '))
print('\n The matrix U is \n')
print('U =', np.array2string(U, prefix='U = '))

The matrix L is 

L = [[10.          0.          0.          0.        ]
     [-1.         10.9         0.          0.        ]
     [ 2.         -0.8         9.5412844   0.        ]
     [ 0.          3.         -0.77981651  7.11057692]]

 The matrix U is 

U = [[ 1.         -0.1         0.2         0.        ]
     [ 0.          1.         -0.0733945   0.27522936]
     [ 0.          0.          1.         -0.08173077]
     [ 0.          0.          0.          1.        ]]


A quick check shows that $A = LU$ is satisfied.

In [12]:
print('LU =', np.array2string(np.dot(L,U), prefix='LU = '), '\n\n','A =', np.array2string(A, prefix='A =  '))

LU = [[10. -1.  2.  0.]
      [-1. 11. -1.  3.]
      [ 2. -1. 10. -1.]
      [ 0.  3. -1.  8.]] 

 A = [[10. -1.  2.  0.]
      [-1. 11. -1.  3.]
      [ 2. -1. 10. -1.]
      [ 0.  3. -1.  8.]]


The system $Ax = b$ can be written as $Ax = (LU)x = L(Ux) = b$. Introducing the vector $z = Ux$, we have $Lz = b$. 

We now solve for $z$ using forward substitution.

In [13]:
# Initialize array for the vector z
z = np.zeros(n)
z[0] = b[0]/L[0, 0]

# Solve Lz = b using forward substitution
for i in range(1, n):
    my_sum = 0
    for j in range(i):
        my_sum += L[i, j] * z[j]
    z[i] = (b[i] - my_sum) / L[i, i]

print('The vector z is \n')
print('z =', z)

The vector z is 

z = [ 0.6         2.34862385 -1.08173077  1.        ]


Again, a quick check shows that $Lz = b$ is satisfied.

In [14]:
print('Lz =', np.dot(L, z), '\n\n', 'b =', b)

Lz = [  6.  25. -11.  15.] 

 b = [6.0, 25.0, -11.0, 15.0]


Now that $z$ is known, the last step is to solve $Ux = z$ for $x$ using backward substitution.

In [15]:
# Initialize array for the vector x
x = np.zeros(n)
x[n-1] = z[n-1]

# Solve Ux = z for x using backward substitution
for i in range(n-2, -1, -1):
    my_sum = 0
    for j in range(n-1, i, -1):
        my_sum = my_sum + U[i, j] * x[j]
    x[i] = (z[i] - my_sum)
    
print('The vector x is \n')
print('x =', x)

The vector x is 

x = [ 1.  2. -1.  1.]


As expected, Crout's method yields the same solution as Doolittle's method. A quick check shows that $Ax = b$ is satisfied.

In [16]:
print('Ax =', np.dot(A, x), '\n\n', 'b =', b, '\n\n', np.dot(A, x)==b)

Ax = [  6.  25. -11.  15.] 

 b = [6.0, 25.0, -11.0, 15.0] 

 [ True  True  True  True]


The last line shows that Crout's method is very accurate as is Doolittle's method.

# Gauss-Seidel

The Gauss-Seidel method is an iterative technique. It requires an initial guess, $x^{(0)}$, for the vector $x$. Then, at each iteration, the elements of the vector $x^{(k)}$ are updated such that
\begin{equation*}
x^{(k + 1)}_i = \frac{1}{a_{ii}} \left[b_i - \sum^{i-1}_{j = 1} \left( a_{ij} x_j^{(k + 1)} \right) - \sum^{n}_{j = i + 1} \left( a_{ij} x^{(k)}_j \right) \right]
\end{equation*}

where $k$ is the iteration number and $i = 1,...,n$.

Here, we are going to solve the system starting with initial guess $x^{(0)} = (0, 0, 0, 0)$ and then $x^{(0)} = (1, 1, 1, 1)$. In order to easily solve the system with both starting points, we are going to implement the Gauss-Seidel method as a function. The function takes as input the matrix $A$, the vector $b$, the initial guess $x^{(0)}$ and the tolerance level at which convergence is achieved, $\epsilon$. The algorithm updates the vector $x$ until convergence is reached. The criterion for convergence is 

\begin{equation*}
\frac{\left| \left| x^{(k + 1)} - x^{(k)} \right| \right|_{\infty}}{\left|\left| x^{(k)} \right| \right|_{\infty}} < \epsilon
\end{equation*}

where $\Vert \,{\gamma}\, \rVert _{\infty} $ is the $l_{\infty}$ norm of the vector $\gamma$.

When convergence is reached, the function returns the vector $x$ and the number of iterations required to reach convergence. At each iteration, the function also prints the vector $x^{(k)}$ along with the $l_{\infty}$ norm criterion. Printing can be turned off by setting the optional argument print_on to False.


In [17]:
def Gauss_Seidel(A, b, guess, tol, print_on=True):
    l_inf = tol + 1
    x = guess
    count = 0
    while l_inf > tol:
        y = x.copy() # keep value of x at previous iterate
        count += 1
        for i in range(n):
            my_sum = 0
            for j in range(n):
                if j != i:
                    my_sum += A[i, j]*x[j]
                    
            x[i] = (b[i]-my_sum)/A[i, i]
        
        # norm criterion
        if max(np.abs(y)) != 0: # Avoid division by zero   
            l_inf = max(np.abs(y - x))/max(np.abs(y)) # l_{\infty} norm criterion
        
        if print_on: # If print_on is True, print output at each iteration
            print('Iteration ' + str(count) + ':')
            print('x =', x)
            if max(np.abs(y)) != 0:
                print('l_inf = ', l_inf, '\n') 
            else:
                print('l_inf = N/A - Division by zero \n')
        
    return x, count

We define the tolerance level for convergence as $\epsilon = 0.0001$.

In [18]:
tol = 0.0001 # epsilon

We first solve the system with initial guess $x^{(0)} = (0, 0, 0, 0)$.

In [19]:
# Initial guess
x = np.zeros(n)

# Solve the system
x, count = Gauss_Seidel(A, b, x, tol)

Iteration 1:
x = [ 0.6         2.32727273 -0.98727273  0.87886364]
l_inf = N/A - Division by zero 

Iteration 2:
x = [ 1.03018182  2.03693802 -1.0144562   0.98434122]
l_inf =  0.18484375000000003 

Iteration 3:
x = [ 1.00658504  2.00355502 -1.00252738  0.99835095]
l_inf =  0.016388814658793216 

Iteration 4:
x = [ 1.00086098  2.00029825 -1.00030728  0.99984975]
l_inf =  0.002856953090344185 

Iteration 5:
x = [ 1.00009128  2.00002134 -1.00003115  0.9999881 ]
l_inf =  0.0003847917873479008 

Iteration 6:
x = [ 1.00000836  2.00000117 -1.00000275  0.99999922]
l_inf =  4.1457869928006095e-05 



With the initial guess $x^{(0)} = (0, 0, 0, 0)$, convergence is reached at the sixth iteration. We can calculate the difference with the actual solution which is $x = (1, 2, -1, 1)$.

In [20]:
print(x - [1, 2, -1, 1])

[ 8.36366133e-06  1.17333627e-06 -2.74507268e-06 -7.83135185e-07]


We now solve the system with initial guess $x^{(0)} = (1, 1, 1, 1)$.

In [21]:
# Initial guess
x = np.ones(n)

# Solve the system
x, count = Gauss_Seidel(A, b, x, tol)

Iteration 1:
x = [ 0.5         2.13636364 -0.88636364  0.96306818]
l_inf =  1.8863636363636362 

Iteration 2:
x = [ 0.99090909  2.01957645 -0.99991736  0.99266916]
l_inf =  0.22978723404255322 

Iteration 3:
x = [ 1.00194112  2.0021833  -1.00090298  0.99906839]
l_inf =  0.008612275598771235 

Iteration 4:
x = [ 1.00039893  2.00020825 -1.00015212  0.99990289]
l_inf =  0.0009864457246365598 

Iteration 5:
x = [ 1.00005125  2.00001731 -1.00001823  0.99999123]
l_inf =  0.00017381979942646743 

Iteration 6:
x = [ 1.00000538  2.00000122 -1.00000183  0.99999931]
l_inf =  2.293582118326794e-05 



With the initial point $x^{(0)} = (1, 1, 1, 1)$, convergence is reached at the sixth iteration. It is the same number of iterations as for the initial point $x^{(0)} = (0, 0, 0, 0)$. We can also calculate the difference with the actual solution.

In [22]:
print(x - [1, 2, -1, 1])

[ 5.37731357e-06  1.22386894e-06 -1.83023074e-06 -6.87729695e-07]


Even though both methods converge within the same number of iterations, the difference between the computed vector and the actual solution is smaller with the initial point $x^{(0)} = (1, 1, 1, 1)$. This suggests that convergence could have been reached faster with this starting point if another tolerance level was chosen.

The returned vector is very close to the actual vector $x$ but they are not exactly equal. Decreasing the tolerance level would increase accuracy. However, the Gauss-Seidel method is less accurate than the Doolittle's and Crout's methods.

# Successive-Over-Relaxation

Successive-Over-Relaxation (SOR) is an iterative method similar to the Gauss-Seidel method. The difference is that an acceleration factor, $\omega$, is used to accelerate convergence. At each iteration, the elements of $x^{(k)}$ are updated such that

\begin{equation*}
x^{(k + 1)}_i = (1 - \omega)x_i^{(k)} + \frac{\omega}{a_{ii}} \left[b_i - \sum^{i-1}_{j = 1} \left( a_{ij} x_j^{(k + 1)} \right) - \sum^{n}_{j = i + 1} \left( a_{ij} x^{(k)}_j \right) \right]
\end{equation*}

The function for SOR is very similar to the one used for the Gauss-Seidel method.

In [23]:
def SOR(A, b, guess, omega, tol, print_on=True):
    l_inf = tol + 1
    x = guess
    count = 0
    while l_inf > tol:
        y = x.copy() # keep value of x at previous iterate as y
        count += 1
        for i in range(n):
            my_sum = 0
            for j in range(n):
                if j != i:
                    my_sum += A[i, j]*x[j]
                    
            x[i] = (1-omega)*y[i] + omega*(b[i]-my_sum)/A[i, i]
        
        # norm criterion
        if max(np.abs(y)) != 0: # Avoid division by zero   
            l_inf = max(np.abs(y - x))/max(np.abs(y)) # l_{\infty} norm criterion
        
        if print_on: # If print_on is true, print output at each iterate
            print('Iteration ' + str(count) + ':')
            print('x =', x)
            if max(np.abs(y)) != 0:
                print('l_inf = ', l_inf, '\n') 
            else:
                print('l_inf = N/A - Division by zero \n')
    
    return x, count

We define the acceleration factor as $\omega = 1.1$ and the tolerance level as $\epsilon = 0.0001$.

In [24]:
omega = 1.1
tol = 0.0001 # epsilon

We now solve the system with initial guess $x^{(0)} = (0, 0, 0, 0)$.

In [25]:
# Initial guess
x = np.zeros(n)

# Solve the system
x, count = SOR(A, b, x, omega, tol)

Iteration 1:
x = [ 0.66        2.566      -1.07294     0.85649575]
l_inf = N/A - Division by zero 

Iteration 2:
x = [ 1.1123068   1.99038796 -1.03425629  1.01360515]
l_inf =  0.22432269875292288 

Iteration 3:
x = [ 0.99524838  1.99297887 -0.99480477  1.00225005]
l_inf =  0.05881186187694755 

Iteration 4:
x = [ 0.99855989  2.00040261 -0.99991091  0.99962117]
l_inf =  0.0037249485598974 

Iteration 5:
x = [ 1.0001687   2.00009917 -1.00007679  0.99998642]
l_inf =  0.0008042432684686864 

Iteration 6:
x = [ 1.00001093  1.99998757 -0.99999759  1.00000682]
l_inf =  7.887918876179545e-05 



With the initial starting point $x^{(0)} = (0, 0, 0, 0)$, convergence is reached in six iterations. This is similar to the results for Gauss-Seidel method. Again, we calculate the difference between the computed vector and the actual solution.

In [26]:
print(x - [1, 2, -1, 1])

[ 1.09315505e-05 -1.24277278e-05  2.41263573e-06  6.81632696e-06]


The difference is bigger than for the Gauss-Seidel method (with both starting points). This suggests that using the acceleration factor $\omega = 1.1$ does not accelerate convergence and even makes convergence slower.

As with the Gauss-Seidel method, the returned vector is close to the true vector $x$ but not equal. Decreasing the tolerance level would increase accuracy. However, iterative methods are less accurate than Doolittle's or Crout's methods. 

# Computational efficiency comparison

In order to assess if a method converges faster, we are going to compare the number of iterations required to converge for various tolerance levels. The considered range for the tolerance level is $[10^{-1}, 10^{-10}]$ with values of $\epsilon$ in the form $10^{-m}, m \in \mathbb{N}$.

First, we use an initial guess $x^{(0)} = (0, 0, 0, 0)$.

In [27]:
for i in range(10):
    x = np.zeros(n) # Initialize x at each step
    tol = 10**(-(i+1))
    count_GS = Gauss_Seidel(A, b, x, tol, print_on=False)[1]
    x = np.zeros(n) # Initialize x for the SOR method
    count_SOR = SOR(A, b, x, omega, tol, print_on=False)[1]

    print('epsilon = ', tol, '\n')
    print('Iterations GS:  ', count_GS, '\n')
    print('Iterations SOR: ', count_SOR, '\n\n')

epsilon =  0.1 

Iterations GS:   3 

Iterations SOR:  3 


epsilon =  0.01 

Iterations GS:   4 

Iterations SOR:  4 


epsilon =  0.001 

Iterations GS:   5 

Iterations SOR:  5 


epsilon =  0.0001 

Iterations GS:   6 

Iterations SOR:  6 


epsilon =  1e-05 

Iterations GS:   7 

Iterations SOR:  7 


epsilon =  1e-06 

Iterations GS:   8 

Iterations SOR:  9 


epsilon =  1e-07 

Iterations GS:   9 

Iterations SOR:  10 


epsilon =  1e-08 

Iterations GS:   10 

Iterations SOR:  11 


epsilon =  1e-09 

Iterations GS:   11 

Iterations SOR:  12 


epsilon =  1e-10 

Iterations GS:   11 

Iterations SOR:  13 




With the initial vector $x^{(0)} = (0, 0, 0, 0)$, both the Gauss-Seidel and SOR methods converge within the same number of iterations for values of $\epsilon$ between $10^{-5}$ and $10^{-1}$. With lower values of $\epsilon$, the SOR method is slower (it needs more iterations to converge).

We now repeat the process with the initial guess $x^{(0)} = (1, 1, 1, 1)$

In [28]:
for i in range(10):
    x = np.ones(n) # Initialize x at each step
    tol = 10**(-(i+1))
    count_GS = Gauss_Seidel(A, b, x, tol, print_on=False)[1]
    x = np.ones(n) # Initialize x for the SOR method
    count_SOR = SOR(A, b, x, omega, tol, print_on=False)[1]

    print('epsilon = ', tol, '\n')
    print('Iterations GS:  ', count_GS, '\n')
    print('Iterations SOR: ', count_SOR, '\n\n')

epsilon =  0.1 

Iterations GS:   3 

Iterations SOR:  3 


epsilon =  0.01 

Iterations GS:   3 

Iterations SOR:  4 


epsilon =  0.001 

Iterations GS:   4 

Iterations SOR:  5 


epsilon =  0.0001 

Iterations GS:   6 

Iterations SOR:  6 


epsilon =  1e-05 

Iterations GS:   7 

Iterations SOR:  7 


epsilon =  1e-06 

Iterations GS:   8 

Iterations SOR:  8 


epsilon =  1e-07 

Iterations GS:   9 

Iterations SOR:  9 


epsilon =  1e-08 

Iterations GS:   10 

Iterations SOR:  11 


epsilon =  1e-09 

Iterations GS:   11 

Iterations SOR:  12 


epsilon =  1e-10 

Iterations GS:   11 

Iterations SOR:  13 




With the initial guess $x^{(0)} = (1, 1, 1, 1)$, the Gauss-Seidel method converges faster for values of epsilon lower than $10^{-8}$ as well as for values between $10^{-2}$ and $10^{-3}$. For other values of $\epsilon$, both methods converge in the same number of iterations. It also appears that the Gauss-Seidel method converges faster with the initial guess $x^{(0)} = (1, 1, 1, 1)$ than with $x^{(0)} = (0, 0, 0, 0)$ when $\epsilon$ is between $10^{-2}$ and $10^{-3}$. For other values of $\epsilon$, the Gauss-Seidel method converges within the same number of iterations with both initial vectors.

Overall, given the problem, the Gauss-Seidel method with initial guess $x^{(0)} = (1, 1, 1, 1)$ seems to converge faster than other iterative methods. 